In [1]:
import requests
from requests.auth import HTTPBasicAuth
import json
import base64
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [3]:
auth = "Bearer OTc3MTkyMjk5NzE4OhR/SuR7NZfuGfgkMKEKAgiRLHZ9"

In [5]:
# Replace BaseURL and project key with your own project
board_url = 'https://issues.apache.org/jira/rest/agile/1.0/board?projectKeyOrId=MESOS'

headers = {
   "Accept": "application/json",
   "Authorization": auth,
}

payload = {
   "maxResults":50 #100, 20
}

# Request data by using GET method and directly get the json output
board = requests.request("GET",board_url,headers=headers, params=payload).json()

# As we only intreseted in the "values" in the board, so we only normalize the board['values']
board_df = pd.json_normalize(board["values"])

# Filter the scrum board type and maintain the pd dataframe formate
board_df = board_df[board_df['type']=='scrum']

# Reset the index for the future convience.
board_df = board_df.reset_index()

# Store the board id into variable board_id
board_id = board_df['id']

#Save the df as csv file. Use your project name to define the file name
board_df.to_csv('MESOS_board.csv') 

In [7]:
headers = {
   "Content-Type": "application/json",
   "Authorization": auth,  
}


payload_sprint = {
'maxResults':2000  #Retrieve at most 2000 records, can be modified based on your own project
}

list1 = [] 

# Loop through the required board, and store the records into the empty list1
for i in board_id:
   sprint_url = 'https://issues.apache.org/jira/rest/agile/1.0/board/' + f'{i}' + '/sprint'
   sprint = requests.request("GET",sprint_url ,headers=headers, params=payload_sprint).json()
   try:
      list1.append(sprint['values'])
   except:
      pass

In [8]:
# Normalize the list and get the inital dataframe, whose record are nested.
sprint_df = pd.json_normalize(list1)
sprint_df.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,"{'id': 45, 'self': 'https://issues.apache.org/...","{'id': 47, 'self': 'https://issues.apache.org/...","{'id': 55, 'self': 'https://issues.apache.org/...","{'id': 69, 'self': 'https://issues.apache.org/...","{'id': 74, 'self': 'https://issues.apache.org/...","{'id': 75, 'self': 'https://issues.apache.org/...","{'id': 81, 'self': 'https://issues.apache.org/...","{'id': 95, 'self': 'https://issues.apache.org/...","{'id': 102, 'self': 'https://issues.apache.org...","{'id': 106, 'self': 'https://issues.apache.org...",...,"{'id': 242, 'self': 'https://issues.apache.org...","{'id': 244, 'self': 'https://issues.apache.org...","{'id': 245, 'self': 'https://issues.apache.org...","{'id': 247, 'self': 'https://issues.apache.org...","{'id': 249, 'self': 'https://issues.apache.org...","{'id': 252, 'self': 'https://issues.apache.org...","{'id': 255, 'self': 'https://issues.apache.org...","{'id': 257, 'self': 'https://issues.apache.org...","{'id': 258, 'self': 'https://issues.apache.org...","{'id': 260, 'self': 'https://issues.apache.org..."
1,"{'id': 217, 'self': 'https://issues.apache.org...","{'id': 219, 'self': 'https://issues.apache.org...","{'id': 234, 'self': 'https://issues.apache.org...","{'id': 285, 'self': 'https://issues.apache.org...","{'id': 323, 'self': 'https://issues.apache.org...","{'id': 324, 'self': 'https://issues.apache.org...","{'id': 326, 'self': 'https://issues.apache.org...","{'id': 327, 'self': 'https://issues.apache.org...","{'id': 331, 'self': 'https://issues.apache.org...","{'id': 332, 'self': 'https://issues.apache.org...",...,None,None,None,None,None,None,None,None,None,None
2,"{'id': 8, 'self': 'https://issues.apache.org/j...","{'id': 12, 'self': 'https://issues.apache.org/...","{'id': 13, 'self': 'https://issues.apache.org/...","{'id': 20, 'self': 'https://issues.apache.org/...","{'id': 22, 'self': 'https://issues.apache.org/...","{'id': 24, 'self': 'https://issues.apache.org/...","{'id': 25, 'self': 'https://issues.apache.org/...","{'id': 30, 'self': 'https://issues.apache.org/...","{'id': 31, 'self': 'https://issues.apache.org/...","{'id': 36, 'self': 'https://issues.apache.org/...",...,"{'id': 175, 'self': 'https://issues.apache.org...","{'id': 180, 'self': 'https://issues.apache.org...","{'id': 188, 'self': 'https://issues.apache.org...","{'id': 192, 'self': 'https://issues.apache.org...","{'id': 214, 'self': 'https://issues.apache.org...","{'id': 215, 'self': 'https://issues.apache.org...","{'id': 216, 'self': 'https://issues.apache.org...","{'id': 217, 'self': 'https://issues.apache.org...","{'id': 219, 'self': 'https://issues.apache.org...","{'id': 222, 'self': 'https://issues.apache.org..."
3,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,"{'id': 47, 'self': 'https://issues.apache.org/...","{'id': 51, 'self': 'https://issues.apache.org/...","{'id': 55, 'self': 'https://issues.apache.org/...","{'id': 56, 'self': 'https://issues.apache.org/...","{'id': 57, 'self': 'https://issues.apache.org/...","{'id': 63, 'self': 'https://issues.apache.org/...","{'id': 69, 'self': 'https://issues.apache.org/...","{'id': 72, 'self': 'https://issues.apache.org/...","{'id': 74, 'self': 'https://issues.apache.org/...","{'id': 75, 'self': 'https://issues.apache.org/...",...,None,None,None,None,None,None,None,None,None,None


In [9]:
#Create an empty dataframe
sprint_result = pd.DataFrame()
sprint_result.head()

# sprint_df.index is the row number. For each row, we need to flatten the records and store them into a dataframe. Our goal is to flatten all these record and generate a single dataframe.
for i in sprint_df.index:
   # Flatten records in a row based on row index
   sprint_df_1 = sprint_df.iloc[i].apply(pd.Series) 
   # Attach corresponding board_id
   sprint_df_1['board_id'] = board_df['id'][i]
   # Create a list called "frame" to tell python we want to contact two dataframs together -- sprint_result and sprint_df_1
   frame = [sprint_result, sprint_df_1]
   #Contact two dataframes
   sprint_result = pd.concat(frame)

In [10]:
# Filter the sprint that totally empty
sprint_result = sprint_result[sprint_result['id'].notna()]
sprint_result.head()

,id,self,state,name,startDate,endDate,completeDate,activatedDate,originBoardId,board_id,goal
0,45.0,https://issues.apache.org/jira/rest/agile/1.0/...,closed,Mesosphere Q4 Sprint 1 10/31,2014-10-19T18:29:03.819Z,2014-10-31T17:29:00.000Z,2014-11-04T00:25:44.904Z,2014-10-19T18:29:03.819Z,62.0,62,NaN
1,47.0,https://issues.apache.org/jira/rest/agile/1.0/...,closed,Mesosphere Q4 Sprint 2 - 11/14,2014-11-04T18:40:28.741Z,2014-11-14T18:40:00.000Z,2014-11-17T18:50:23.756Z,2014-11-04T18:40:28.741Z,62.0,62,NaN
2,55.0,https://issues.apache.org/jira/rest/agile/1.0/...,closed,Mesosphere Q4 Sprint 3 - 12/7,2014-11-19T17:58:44.739Z,2014-12-07T17:58:00.000Z,2015-01-15T21:15:18.091Z,2014-11-19T17:58:44.739Z,62.0,62,NaN
3,69.0,https://issues.apache.org/jira/rest/agile/1.0/...,closed,Mesosphere Q1 Sprint 1 - 1/23,2015-01-15T21:16:49.334Z,2015-01-23T21:16:00.000Z,2015-01-23T16:40:01.808Z,2015-01-15T21:16:49.334Z,62.0,62,NaN
4,74.0,https://issues.apache.org/jira/rest/agile/1.0/...,closed,Mesosphere Q1 Sprint 2 - 2/6,2015-01-23T16:54:34.977Z,2015-02-06T16:54:00.000Z,2015-02-06T16:22:48.222Z,2015-01-23T16:54:34.977Z,62.0,62,NaN


In [11]:
# Reset the index
sprint_result = sprint_result.reset_index()
# Drop the origianl index column
sprint_result = sprint_result.drop(columns=['index'])
sprint_result.head()

,id,self,state,name,startDate,endDate,completeDate,activatedDate,originBoardId,board_id,goal
0,45.0,https://issues.apache.org/jira/rest/agile/1.0/...,closed,Mesosphere Q4 Sprint 1 10/31,2014-10-19T18:29:03.819Z,2014-10-31T17:29:00.000Z,2014-11-04T00:25:44.904Z,2014-10-19T18:29:03.819Z,62.0,62,NaN
1,47.0,https://issues.apache.org/jira/rest/agile/1.0/...,closed,Mesosphere Q4 Sprint 2 - 11/14,2014-11-04T18:40:28.741Z,2014-11-14T18:40:00.000Z,2014-11-17T18:50:23.756Z,2014-11-04T18:40:28.741Z,62.0,62,NaN
2,55.0,https://issues.apache.org/jira/rest/agile/1.0/...,closed,Mesosphere Q4 Sprint 3 - 12/7,2014-11-19T17:58:44.739Z,2014-12-07T17:58:00.000Z,2015-01-15T21:15:18.091Z,2014-11-19T17:58:44.739Z,62.0,62,NaN
3,69.0,https://issues.apache.org/jira/rest/agile/1.0/...,closed,Mesosphere Q1 Sprint 1 - 1/23,2015-01-15T21:16:49.334Z,2015-01-23T21:16:00.000Z,2015-01-23T16:40:01.808Z,2015-01-15T21:16:49.334Z,62.0,62,NaN
4,74.0,https://issues.apache.org/jira/rest/agile/1.0/...,closed,Mesosphere Q1 Sprint 2 - 2/6,2015-01-23T16:54:34.977Z,2015-02-06T16:54:00.000Z,2015-02-06T16:22:48.222Z,2015-01-23T16:54:34.977Z,62.0,62,NaN


In [12]:
# Save as csv file
sprint_result.to_csv('MESOS_sprint.csv')

In [13]:
headers = {
   "Content-Type": "application/json",
   "Authorization": auth,  
}

payload_issues = {
'maxResults':3000
}

list1 = []
# Loop through the required board, and store the records into the empty list1
for i in board_id:
   issues_url = 'https://issues.apache.org/jira/rest/agile/1.0/board/' + f'{i}' + '/issue'
   issues = requests.request("GET",issues_url ,headers=headers, params=payload_issues).json()
   try:
      list1.append(issues['issues'])
   except:
      pass

In [14]:
# Normalize the list and get the inital dataframe, whose record are nested.
issue_df = pd.json_normalize(list1)
issue_df.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,"{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...",...,None,None,None,None,None,None,None,None,None,None
1,"{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...",...,None,None,None,None,None,None,None,None,None,None
2,"{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...",...,"{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation..."
3,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,"{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...",...,None,None,None,None,None,None,None,None,None,None


In [15]:
issue_result = pd.DataFrame()

for i in issue_df.index:
   issue_df_1 = issue_df.iloc[i].apply(pd.Series)
   issue_df_1['board_id'] = board_df['id'][i]
   frame = [issue_result, issue_df_1]
   issue_result = pd.concat(frame)

issue_result = issue_result.reset_index()
issue_result = issue_result.drop(columns=['index'])

In [17]:
#Only select issues that have story points
issue_result = issue_result[issue_result['fields.customfield_12310293'].notna()]
issue_result.head()

,expand,id,self,key,fields.fixVersions,fields.resolution.self,fields.resolution.id,fields.resolution.description,fields.resolution.name,fields.customfield_12312322,...,board_id,fields.sprint.id,fields.sprint.self,fields.sprint.state,fields.sprint.name,fields.sprint.startDate,fields.sprint.endDate,fields.sprint.activatedDate,fields.sprint.originBoardId,fields.sprint.goal
0,"operations,versionedRepresentations,editmeta,c...",13217940,https://issues.apache.org/jira/rest/agile/1.0/...,MESOS-9609,[{'self': 'https://issues.apache.org/jira/rest...,https://issues.apache.org/jira/rest/api/2/reso...,1,A fix for this issue is checked into the tree ...,Fixed,NaN,...,62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"operations,versionedRepresentations,editmeta,c...",13128691,https://issues.apache.org/jira/rest/agile/1.0/...,MESOS-8400,[],NaN,NaN,NaN,NaN,NaN,...,62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,"operations,versionedRepresentations,editmeta,c...",13193889,https://issues.apache.org/jira/rest/agile/1.0/...,MESOS-9352,[],https://issues.apache.org/jira/rest/api/2/reso...,5,"All attempts at reproducing this issue failed,...",Cannot Reproduce,NaN,...,62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,"operations,versionedRepresentations,editmeta,c...",13152589,https://issues.apache.org/jira/rest/agile/1.0/...,MESOS-8790,[],NaN,NaN,NaN,NaN,NaN,...,188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1001,"operations,versionedRepresentations,editmeta,c...",13108424,https://issues.apache.org/jira/rest/agile/1.0/...,MESOS-8069,[],NaN,NaN,NaN,NaN,NaN,...,188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
#Check the dataframe shape
issue_result.shape

(709, 242)

In [19]:
#Save as csv file
issue_result.to_csv('MESOS_issues.csv')